In [4]:
import os
import matplotlib.pyplot as plt
import numpy as np
#from tqdm.auto import tqdm
from tqdm import tqdm
plt.ioff();
try:
    maindir
except NameError:
    maindir = os.path.dirname(os.getcwd())
for module in tqdm(os.listdir(maindir + '\\testing_modules')):
    %run -i "{maindir}\\testing_modules\\{module}"
base = os.path.dirname(os.path.dirname(maindir)) + '\\'
plt.ion();
np.set_printoptions(suppress=True)
plt.close('All')

100%|██████████| 20/20 [00:00<00:00, 414.44it/s]


In [5]:
%matplotlib qt
import skimage.io as io
import xarray as xr
from collections import OrderedDict
import dask
from tqdm.dask import TqdmCallback

In [376]:
scanid = 149566
filedir = '''D:\\Musterman_postdoc\\20231109_Musterman\\'''
filedir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Data\\Condensed Datasets\\'''
h5_filename = f'scan{scanid}_xrd.h5'

test = XRDMap.from_h5(h5_filename, wd=filedir)

Loading data from h5 file...
Loading most recent images (raw_images)...

done!
Loading reciprocal positions...done!
Loading saved phases...done!
XRD Map loaded!


In [5]:
scanid = 149248
#filedir = '''D:\\Musterman_postdoc\\20231109_Musterman\\'''
filedir = '''D:\\Musterman_postdoc\\20231030_Musterman\\'''
filedir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Data\\Condensed Datasets\\'''
h5_filename = f'scan{scanid}_xrd.h5'

test = XRDMap.from_h5(h5_filename, wd=filedir)

Loading data from h5 file...
Loading most recent images (raw_images)...

done!
Loading reciprocal positions...done!
Loading saved phases...done!
Loading reflection spots...done!
XRD Map loaded!


In [4]:
scanid = 149248
filedir = '''D:\\Musterman_postdoc\\20231030_Musterman\\'''
filedir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Data\\Condensed Datasets\\'''
map_filename = f'scan{scanid}_dexela_xrd.tif'
h5_filename = f'scan{scanid}_xrd.h5'

test = XRDMap.from_image_stack(map_filename, wd=filedir, energy=15, scanid=scanid)
dark_field = io.imread(f'{filedir}dexela_df_avg.tif')
test.set_calibration('scan149256_dexela_calibration.poni')

Loading images...done!
Writing images to h5...done!


In [6]:
scanid = 149566
filedir = '''D:\\Musterman_postdoc\\20231109_Musterman\\'''
filedir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Data\\Condensed Datasets\\'''
map_filename = f'scan{scanid}_dexela_xrd.tif'
h5_filename = f'scan{scanid}_xrd.h5'

test = XRDMap.from_image_stack(map_filename, wd=filedir, energy=15, scanid=scanid)
dark_field = io.imread(f'{filedir}dexela_df_avg.tif')
test.set_calibration('scan149562_dexela_calibration.poni')

Loading images...done!
Writing images to h5...done!


In [8]:
dark_field = io.imread(f'{filedir}dexela_df_avg.tif')
scaled_df = dark_field * np.median(test.map.min_image) / np.median(dark_field)

In [9]:
scaled_df = dark_field * np.median(test.map.min_image) / np.median(dark_field)
test.map.correct_dark_field(dark_field=scaled_df)
test.map.correct_outliers()
test.map.images[test.map.images < 0] = 0
# Much faster for smaller images. Should not matter when this is applied...
test.map.normalize_scaler()

Correcting dark-field...done!
Finding and correcting image outliers...

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [00:00<00:00, 66.29it/s]

done!
No scaler array given. Approximating with image medians...
Normalize image scalers...

done!


In [9]:
test.map.interactive_map(img_vmin=0, img_vmax=5)

AttributeError: 'ImageMap' object has no attribute 'tth'

In [40]:
cif_dir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Literature\\CIF\\'''
test.clear_phases()
for filename, phase_name in zip(['AMCSD\\Hematite_0017806.cif',
                                 'AMCSD\\Iron-alpha_0011214.cif',
                                 'AMCSD\\Iron-beta_0011215.cif',
                                 'AMCSD\\Iron-delta_0011216.cif',
                                 'AMCSD\\Iron_0011146.cif',
                                 'AMCSD\\Cementite_0013523.cif',
                                 'AMCSD\\Zincite_0011555.cif'],
                                ['Hematite', 'Iron-ferrite', 'Iron-beta',
                                 'Iron-delta', 'Iron-austenite', 'Cementite',
                                 'Zincite']):
    
    test.load_phase(filename, filedir=cif_dir, phase_name=phase_name)

In [10]:
cif_dir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Literature\\CIF\\'''
test.clear_phases()
test.load_phase('Barker\\1521772COD.cif', filedir=cif_dir, phase_name="LiNbO3 161")
test.load_phase('Barker\\LiNbO3.cif', filedir=cif_dir, phase_name="LiNbO3 148")
test.load_phase('AMCSD\\Platinum_0011157.cif', filedir=cif_dir, phase_name="Platinum")
test.load_phase('AMCSD\\Calcite_0000984.cif', filedir=cif_dir, phase_name="Calcite")
test.load_phase('AMCSD\\Aluminum_0011136.cif', filedir=cif_dir, phase_name="Aluminum")
test.load_phase('AMCSD\\Zincite_0011555.cif', filedir=cif_dir, phase_name="Zincite")
test.load_phase('AMCSD\\Gold_0011140.cif', filedir=cif_dir, phase_name="Gold")
test.load_phase('AMCSD\\Chromium_0011209.cif', filedir=cif_dir, phase_name="Chromium BCC")
test.load_phase('AMCSD\\Chromium_0011170.cif', filedir=cif_dir, phase_name="Chromium HCP")
test.load_phase('AMCSD\\Chromium_0011144.cif', filedir=cif_dir, phase_name="Chromium FCC")

XU.materials: Warning: element O used instead of O-2
XU.materials: Warning: element Nb used instead of Nb+5
XU.materials: Warning: element Li used instead of Li+1


In [11]:
test.select_phases(remove_less_than=0.1)

In [10]:
test.calibrate_images()

Calibrating images...

100%|██████████| 36/36 [00:08<00:00,  4.26it/s]


done!
Writing reciprocal positions...
done!
Acquring and writing calibration mask...
done!


In [224]:
def masked_filter(image, mask, filter, **kwargs):
    nan_image = np.copy(image)
    nan_image[~mask] = np.nan

    zero_image = np.copy(image)
    zero_image[~mask] = 0 # should be redundant
    gauss_zero = filter(zero_image, **kwargs)

    div_image = 0 * np.copy(nan_image) + 1
    div_image[~mask] = 0
    gauss_div = filter(div_image, **kwargs)

    overlap = gauss_zero / gauss_div
    overlap[~mask] = 0

    return overlap

In [141]:
def masked_gaussian_background(image, mask, sigma, return_background=False):
    nan_image = np.copy(image)
    nan_image[~mask] = np.nan

    zero_image = np.copy(image)
    zero_image[~mask] = 0 # should be redundant
    gauss_zero = gaussian_filter(zero_image, sigma=sigma)

    div_image = 0 * np.copy(nan_image) + 1
    div_image[~mask] = 0
    gauss_div = gaussian_filter(div_image, sigma=sigma)

    overlap = gauss_zero / gauss_div
    overlap[~mask] = 0
    
    if return_background:
        return overlap
    else:
        return image - overlap
    

In [354]:
def masked_gaussian_background(imagemap, sigma, return_background=False):
    map_shape = imagemap.images.shape[:2]
    image_shape = imagemap.images.shape[2:]

    mask = np.empty_like(imagemap.images, dtype=np.bool_)
    mask[:, :] = imagemap.calibration_mask

    nan_images = np.copy(imagemap.images)
    nan_images[~mask] = np.nan

    zero_image = np.copy(imagemap.images)
    zero_image[~mask] = 0 # should be redundant
    gauss_zero = gaussian_filter(zero_image, sigma=(0, 0, sigma, sigma))

    div_image = 0 * np.copy(nan_images) + 1
    div_image[~mask] = 0
    gauss_div = gaussian_filter(div_image, sigma=(0, 0, sigma, sigma))

    overlap = gauss_zero / gauss_div
    overlap[~mask] = 0
    
    if return_background:
        return overlap
    else:
        return imagemap.images - overlap

In [358]:
gauss_bkg = masked_gaussian_background(test.map, sigma=100, return_background=True)

In [359]:
new_images = test.map.images - gauss_bkg

In [369]:
test.map.images = rescale_array(test.map.images, upper=100, lower=0)

In [374]:
map_noise = estimate_map_noise(test.map)

In [375]:
spot_search(test.map.images[2, 5], multiplier=100, bkg_noise=map_noise, plotme=True)

(array([], shape=(0, 2), dtype=int32),
 array([[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]]),
 array([[0.70871466, 0.70871466, 0.70871466, ..., 0.70871466, 0.70871466,
         0.70871466],
        [0.70871466, 0.70871466, 0.70871466, ..., 0.70871466, 0.70871466,
         0.70871466],
        [0.70871466, 0.70871466, 0.70871466, ..., 0.70871466, 0.70871466,
         0.70871466],
        ...,
        [0.70871466, 0.70871466, 0.70871466, ..., 0.70871466, 0.70871466,
         0.70871466],
        [0.70871466, 0.70871466, 0.70871466, ..., 0.70871466, 0.70871466,
         0.70871466],
        [0.70871466, 0.70871466, 0.70871466, ..., 0.70871466, 0.70871466,
         0.70871466]], dtype=float32)

In [280]:
i = np.random.randint(test.map.map_shape[0])
j = np.random.randint(test.map.map_shape[1])

image = masked_gaussian_background(test.map.images[2, 5], test.map.calibration_mask, sigma=200)
test.map.plot_image(image)

In [284]:
i = np.random.randint(test.map.map_shape[0])
j = np.random.randint(test.map.map_shape[1])

image = masked_gaussian_background(test.map.images[i, j], test.map.calibration_mask, sigma=200, return_background=False)
test.map.plot_image(image)

In [140]:
test.map.plot_image(test.map.images[0, 0] - Z)

In [191]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import RectBivariateSpline

skip = 20
image = test.map.images[i, j][::skip, ::skip]


func = RectBivariateSpline(test.tth[::skip], test.chi[::skip], image.T, s=1000)

znew = func(test.tth, test.chi).T
znew[~test.map.calibration_mask] = 0

test.map.plot_image(test.map.images[i, j] - znew)

In [39]:
test.map.plot_image(test.map.images[0, 0] - znew)

In [108]:
def poly_bkg(xy, *args):
    order = (len(args) - 1) // 2

    x = np.asarray(xy[0])
    y = np.asarray(xy[1])

    z = 0
    for power in range(order):
        #print(order-power)
        z += args[2 * power] * x ** (order - power)
        z += args[2 * power + 1] * y ** (order - power)
    
    z += args[-1] # offset at end

    return z

In [113]:
x_coords, y_coords = np.meshgrid(test.tth, test.chi[::-1])
x_fit = x_coords[test.map.calibration_mask]
y_fit = y_coords[test.map.calibration_mask]
z_fit = test.map.med_image[test.map.calibration_mask]

order = 10
p0 = np.ones(order * 2 + 1) * 1e-5

popt, pcov = curve_fit(poly_bkg, [x_fit, y_fit], z_fit, p0=p0)

In [114]:
new_z = poly_bkg([x_coords, y_coords], *popt)
new_z[~test.map.calibration_mask] = 0

In [115]:
test.map.plot_image(new_z)
test.map.plot_image(test.map.images[0, 0] - new_z)

In [221]:
def bruckner_background(image, size, iterations, return_background=False):
    # Cannot currently handle Nans...
    from scipy.ndimage import uniform_filter

    old_image = np.copy(image)

    for i in range(iterations):
        avg_image = uniform_filter(old_image, size=size)
        min_image = np.min(np.array([image, avg_image]), axis=0)
        old_image = min_image

    if return_background:
        return min_image
    else:
        return image - min_image

In [328]:
def masked_bruckner_background(image, size, iterations, mask, return_background=False):
    # Cannot currently handle Nans...
    from scipy.ndimage import uniform_filter
    old_image = np.copy(image)
    init_min = np.min(old_image)
    init_avg = np.mean(old_image)
    init_thresh = init_avg + 2 * (init_avg - init_min)
    init_mask = (old_image >= init_thresh)
    old_image[init_mask] = init_thresh
    # Initially filter to help with noise
    old_image = masked_filter(old_image, mask, uniform_filter, size=size)

    for i in range(iterations):
        avg_image = masked_filter(old_image, mask, uniform_filter, size=size)
        #avg_image = uniform_filter(old_image, size=size)
        #min_image = np.min(np.array([old_image, avg_image]), axis=0)
        min_image = np.min(np.array([old_image, avg_image]), axis=0)
        
        if np.max(np.abs(old_image - min_image)) > 1e-3:
            old_image = np.copy(min_image)
        else:
            print(f'Background converged after {i + 1} iterations!')
            break

    if return_background:
        return min_image
    else:
        return image - min_image

In [339]:
tic()
test.map.plot_image(masked_bruckner_background(test.map.images[2, 5], 50, 100,
                                               test.map.calibration_mask,
                                               return_background=False))
toc()
tic()
test.map.plot_image(masked_gaussian_background(test.map.images[2, 5],
                                               test.map.calibration_mask,
                                               sigma=100, return_background=False))
toc()


dt = 7.562 sec

dt = 1.907 sec


In [340]:
gauss_bkg = gaussian_filter(test.map.images, sigma=(0, 0, 100, 100))

In [341]:
test.map.plot_image(gauss_bkg[0, 0])

In [278]:
test.map.plot_image(bruckner_background(test.map.images[2, 5], 30, 50, return_background=True))

In [223]:
test.map.plot_image(test.map.images[2, 5])

In [210]:
from scipy.ndimage import uniform_filter

image = test.map.images[2, 5]
results = uniform_filter(image, size=10)
results = np.min(np.array([image, results]), axis=0)
test.map.plot_image(results)

In [206]:
test.map.plot_image(results)

In [201]:
bins = np.ones((1, 3))
results = np.convolve(test.map.images[2, 5], bins, mode='same') / len(bins)

ValueError: object too deep for desired array

In [12]:
test.map.remove_background(background=test.map.images[12, 10])

Removing background...done!


In [45]:
test.map.plot_image(test.map.images[12, 10])

In [13]:
arr_min = np.min(test.map.images)
arr_max = np.max(test.map.images)

In [14]:
def iter_rescale_array(arr, lower=0, upper=1, arr_min=None, arr_max=None):
    # Works for arrays of any size including images!
    if arr_min is None:
        arr_min = np.min(arr)
    if arr_max is None:
        arr_max = np.max(arr)
    ext = upper - lower

    for index in tqdm(range(np.multiply(*arr.shape[:2])), desc='Scheduling...'):
        indices = np.unravel_index(index, arr.shape[:2])
        arr[indices] = lower + ext * ((arr[indices] - arr_min) / (arr_max - arr_min))

In [15]:
iter_rescale_array(test.map.images, lower=0, upper=100, arr_min=arr_min, arr_max=arr_max)

Scheduling...: 100%|██████████| 3721/3721 [05:23<00:00, 11.51it/s]


In [18]:
test.map.plot_image()

In [73]:
def iter_rescale_array(arr, lower=0, upper=1, arr_min=None, arr_max=None):
    # Works for arrays of any size including images!
    if arr_min is None:
        arr_min = np.min(arr)
    if arr_max is None:
        arr_max = np.max(arr)
    ext = upper - lower

    @dask.delayed
    def delayed_rescale_array(arr, lower, ext, arr_min, arr_max, indices):
        arr[indices] = lower + ext * ((arr[indices] - arr_min) / (arr_max - arr_min))
        return arr[indices]

    delayed_list = []
    for index in tqdm(range(np.multiply(*arr.shape[:2])), desc='Scheduling...'):
        indices = np.unravel_index(index, arr.shape[:2])

        delayed_list.append(delayed_rescale_array(arr[indices], lower, ext, arr_min, arr_max, indices))

    #return delayed_list
    with TqdmCallback(desc='Computing...', tqdm_class=tqdm):
        dask.compute(*delayed_list)

In [75]:
test.map.plot_image()

In [ ]:
#nan_med = test.map.med_image
nan_med = median_image
nan_med[~test.map.calibration_mask] = np.nan
ball_bkg = rolling_ball(gaussian_filter(nan_med, sigma=2), radius=50)
ball_bkg[~test.map.calibration_mask] = 0
test.map.remove_background(background=ball_bkg * test.map.calibration_mask)
test.map.images = rescale_array(test.map.images, lower=0, upper=100)

In [107]:
test.map.update_map_title()
test.map.save_images(units=test.map.calib_unit, labels=['x_ind', 'y_ind', 'chi_ind', 'tth_ind'])

In [19]:
map_noise = estimate_map_noise(test.map)

In [36]:
i = np.random.randint(test.map.map_shape[0])
j = np.random.randint(test.map.map_shape[1])

out = spot_search(test.map.images[i, j], bkg_noise=map_noise, multiplier=3, plotme=True)

In [19]:
test.map.plot_image(out[1] * out[2])

In [37]:
test.find_spots(multiplier=3)

Characterizing spots...: 100%|██████████| 22193/22193 [09:00<00:00, 41.08it/s]


In [38]:
test.fit_spots(GaussianFunctions, max_dist=0.75)

Fitting spots in blobs...: 100%|██████████| 19175/19175 [10:19<00:00, 30.95it/s]


Successfully fit 16396 / 22193 spots ( 73.9 % ).


In [40]:
plot_map = np.zeros(test.map.map_shape)
plot_map[:] = np.nan

for image_index in range(test.map.num_images):
    indices = np.unravel_index(image_index, test.map.map_shape)
    pixel_df = test.spots[(test.spots['map_x'] == indices[0])
                          & (test.spots['map_y'] == indices[1])]
    
    if len(pixel_df) < 1:
        plot_map[indices] = 0
    else:
        plot_map[indices] = np.sum(pixel_df['guess_int'])
        #plot_map[indices] = np.sum(pixel_df['fit_amp'])
        #plot_map[indices] = np.mean(pixel_df['fit_x0'])
        #plot_map[indices] = len(pixel_df)


fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

im = ax.imshow(plot_map)
fig.colorbar(im, ax=ax)
ax.set_title('Spot Value')

plt.show()

In [67]:
test.map.plot_image()

In [78]:
[test.phases[phase].get_hkl_reflections(tth_range=(np.min(test.tth), np.max(test.tth))) for phase in test.phases];

pot_tth = np.concatenate([test.phases[key].reflections['tth'] for key in test.phases])
pot_hkl = np.concatenate([test.phases[key].reflections['hkl'] for key in test.phases])
pot_phase = np.concatenate([[key,] * len(test.phases[key].reflections['tth']) for key in test.phases])

search_distance = 0.5 # in degrees
this_phase = 'Iron-ferrite'

plot_map = np.zeros(test.map.map_shape)
plot_map[:] = 0

for image_index in range(test.map.num_images):
    indices = np.unravel_index(image_index, test.map.map_shape)
    pixel_df = test.spots[(test.spots['map_x'] == indices[0])
                          & (test.spots['map_y'] == indices[1])]
    
    if len(pixel_df) < 1:
        plot_map[indices] = 0
    else:
        for spot_index in range(len(pixel_df)):
            min_arr = np.abs(pot_tth - pixel_df.iloc[spot_index]['guess_cenx'])
            #min_arr = np.array([np.abs(x - pot_tth) for x in pixel_df.loc[spot_index, 'fit_x0']])
            min_arr[min_arr > search_distance] = np.nan
            if np.all(np.isnan(min_arr)):
                continue
            #min_index = [np.nanargmin(x) if np.any(~np.isnan(x)) else None for x in min_arr]
            min_index = np.nanargmin(min_arr)

            if pot_phase[min_index] == this_phase:
                plot_map[indices] += pixel_df.iloc[spot_index]['guess_int']

        #plot_map[indices] = np.sum(pixel_df['guess_int'])
        #plot_map[indices] = np.sum(pixel_df['fit_amp'])
        #plot_map[indices] = np.mean(pixel_df['fit_x0'])
        #plot_map[indices] = len(pixel_df)


fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

im = ax.imshow(plot_map)
fig.colorbar(im, ax=ax)
ax.set_title(this_phase)

plt.show()

In [438]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

image = test.map.composite_image

tth = test.tth
intensity = np.sum(test.map.composite_image, axis=0)
pixels = np.sum(test.map.calibration_mask, axis=0)

x, bins, _ = ax.hist(test.spots['guess_cenx'].values, bins=2000, weights=test.spots['guess_height'].values, label='spot center', density=True)
ax.plot(tth, rescale_array(intensity / pixels, lower=np.min(x), upper=np.max(x)), label='integrated intensity')
ax.legend()

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

image = test.map.composite_image

tth = test.tth
intensity = np.sum(test.map.composite_image, axis=0)
pixels = np.sum(test.map.calibration_mask, axis=0)

x, bins, _ = ax.hist(test.spots['fit_x0'].values, bins=2500, label='spot center', density=True)
ax.plot(tth, rescale_array(intensity / pixels, lower=np.min(x), upper=np.max(x)), label='integrated intensity')
ax.legend()

plt.show()

In [42]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

image = test.map.composite_image

tth = test.tth
x, bins, _ = ax.hist(test.spots['fit_x0'].values, bins=2000, label='fit', range=test.extent[:2], alpha=0.5)
x, bins, _ = ax.hist(test.spots['guess_cenx'].values, bins=2000, label='integrated', range=test.extent[:2], alpha=0.5)

#ax.plot(tth, rescale_array(intensity / pixels, lower=np.min(x), upper=np.max(x)), label='integrated intensity')
ax.legend()

plt.show()

In [323]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

image = test.map.composite_image
bins = 2500

guess_val = test.spots['guess_cenx'].values
fit_val = test.spots['fit_x0'].values

tth = test.tth
ext = [np.nanmin([guess_val, fit_val]),
       np.nanmax([guess_val, fit_val])]
x, bins, _ = ax.hist(fit_val, bins=bins, label='fit', alpha=0.5, range=ext)
x, bins, _ = ax.hist(guess_val, bins=bins, label='integrated', alpha=0.5, range=ext)

#ax.plot(tth, rescale_array(intensity / pixels, lower=np.min(x), upper=np.max(x)), label='integrated intensity')
ax.legend()

plt.show()

In [104]:
thickness = 0.6
corrected_tth = test.tth + (thickness * np.sin(np.radians(test.tth)))
rad = np.radians(corrected_tth / 2)
intensity = np.sum(test.map.composite_image * (np.sin(rad) * np.sin(2 * rad)), axis=0)
intensity = np.sum(test.map.composite_image, axis=0)
pixels = np.sum((test.map.composite_image != 0), axis=0)
phase_vals = phase_selector(intensity / pixels, list(test.phases.values()), corrected_tth)
test.map.images = rescale_array(test.map.images, lower=0, upper=100) # Remove negative values. They break spot stats

In [170]:
bkg_noise = estimate_map_noise(test)

In [171]:
i = np.random.randint(test.map.map_shape[0])
j = np.random.randint(test.map.map_shape[1])

out = spot_search(test.map.images[i, j], bkg_noise, multiplier=10, sigma=3, plotme=True)

In [288]:
test.map.plot_image(test.map.images[i, j] * out[1])

In [175]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

bins = 1000
key = 'guess_stdy'
#ax.hist(stat_dict[key], bins=bins, density=True)
ax.hist(stat_dict[key], bins=bins, weights=stat_dict['guess_int'], density=True)

plt.show()

In [133]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

image = test.map.composite_image

tth = test.tth
intensity = np.sum(test.map.composite_image, axis=0)
pixels = np.sum(test.map.calibration_mask, axis=0)

x, bins, _ = ax.hist(stat_dict['guess_cenx'], bins=2000, weights=stat_dict['guess_height'], label='spot center', density=True)
ax.plot(tth, rescale_array(intensity / pixels, lower=np.min(x), upper=np.max(x)), label='integrated intensity')
ax.legend()

plt.show()

In [139]:
stat_df = pd.DataFrame.from_dict(stat_dict)

In [162]:
plot_map = np.zeros(test.map.map_shape)

for pixel_index in range(test.map.num_images):
    indices = np.unravel_index(pixel_index, test.map.map_shape)
    pixel_df = stat_df.loc[(stat_df['map_x'] == indices[0]) & (stat_df['map_y'] == indices[1])]

    val = np.mean(pixel_df['guess_int'])

    plot_map[indices] = val

In [163]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

im = ax.imshow(np.array(plot_map).reshape(test.map.images.shape[:2]))
fig.colorbar(im, ax=ax)
ax.set_title('Total Reflection Intensity')

plt.show()

In [43]:
[test.phases[phase].get_hkl_reflections() for phase in test.phases];

In [38]:
new_out = [np.asarray(x) for x in stat_list]

pot_tth = np.concatenate([test.phases[key].reflections['tth'] for key in test.phases])
pot_hkl = np.concatenate([test.phases[key].reflections['hkl'] for key in test.phases])
pot_phase = np.concatenate([[key,] * len(test.phases[key].reflections['tth']) for key in test.phases])

plot_value = []
search_distance = 0.5 # in degrees
phase = 'Iron-austenite'

for spots in new_out:
    if len(spots.shape) > 1:
        #val = np.sum(spots[:, 5])
        min_arr = np.array([np.abs(x - pot_tth) for x in spots[:, 5]])
        min_arr[min_arr > search_distance] = np.nan
        min_index = [np.nanargmin(x) if np.any(~np.isnan(x)) else None for x in min_arr]
        
        val = 0
        for spot_idx, index in enumerate(min_index):
            if index is None:
                continue
            else:
                #raise Exception
                if pot_phase[index] == phase:
                    val += spots[spot_idx, -1]
        
    else:
        val = 0
    
    plot_value.append(val)

In [39]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

im = ax.imshow(np.array(plot_value).reshape(test.map.images.shape[:2]))
fig.colorbar(im, ax=ax)
ax.set_title(f'{phase} Total Intensity')

plt.show()

In [292]:
selected_images = np.median(np.copy(test.map.images[::5, ::5]), axis=(0, 1)) # Randomize for less that certain number of images...
bkg_noise = np.std(selected_images[selected_images != 0])

i = np.random.randint(test.map.images.shape[0])
j = np.random.randint(test.map.images.shape[1])
#i, j = 0, 1
image = test.map.images[i, j]

spot_dict = find_spots(image, bkg_noise,
                               multiplier=10, sigma=3,
                               tth=test.tth, chi=test.chi, plotme=True)
print(len(spot_dict['guess_height']))

In [ ]:
def dask_find_spots(pixel):
    spot_dict, mask = find_spots(pixel, bkg_noise,
                               multiplier=10, sigma=3,
                               tth=test.tth, chi=test.chi)
    return spot_dict, mask

iterable = test.map.images.reshape(test.map.num_images, len(test.chi), len(test.tth))
spot_dict_lst = parallel_loop(dask_find_spots, iterable)

In [284]:
iterable = test.map.images.reshape(test.map.num_images, len(test.chi), len(test.tth))

spot_dict_lst = []
mask_lst = []
for pixel in iterable:
    spot_dict, mask = find_spots(pixel, bkg_noise,
                               multiplier=10, sigma=3,
                               tth=test.tth, chi=test.chi)
    spot_dict_lst.append(spot_dict)
    mask_lst.append(mask)

In [22]:
elements_folder = f'output_tiff_scan2D_{scanid}_xs_sum8ch\\'

element_maps = {}
for element in os.listdir(f'{filedir}{elements_folder}'):
    el_im = io.imread(f'{filedir}{elements_folder}{element}')
    element_maps[str(element[:-5])] = el_im

In [23]:
key = 'detsum_Fe_K'
#interactive_dynamic_2d_plot(proc_map, range(121), range(121), img_vmin=10, img_vmax=100, display_map=element_maps['detsum_Pt_L'], map_vmin=1e4, map_vmax=1.1e4)
interactive_dynamic_2d_plot(test.map.images, range(test.map.images.shape[0]), range(test.map.images.shape[1]), display_map=element_maps[key], display_title=key)